# Syntax tutorial

This notebook is a terse tutorial walkthrough of the syntax of the `guidance` language (which is based on the Handlebars templating language).

In [1]:
from promptitude import guidance

# we will use GPT-4o-mini for most of the examples in this tutorial
guidance.llm = guidance.llms.OpenAI("gpt-4o-mini")
# guidance.llm = guidance.llms.Anthropic("claude-3-5-haiku-20241022")

## Basic templating

### Single variable

In [2]:
program = guidance('''What is {{example}}?''')

# this program has not been executed yet, so it still has the template placeholder in it
program 

Stop program What is {{example}} ?

In [3]:
# when we execute the program (by calling it) template placeholders are filled in
# note that keyword arguments to the program become variables in the template namespace
executed_program = program(example='truth')

Stop program What is truth ?

In [4]:
# all the variables used by the program are returned as part of the executed program
executed_program['example']

'truth'

### Lists and objects

In [5]:
# define some variables we will use in the guidance program
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [
    {'name': 'truth', 'description': 'the state of being the case'},
    {'name': 'love', 'description': 'a strong feeling of affection'}
]

# we can use the `each` block to iterate over a list
program = guidance('''
List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace either before or after a tag, depending on where you place it}}
{{/each~}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each~}}
''')

executed_program = program(people=people, ideas=ideas)

Stop program 
List of people:
 - John 
- Mary 
- Bob 
- Alice 
 List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

### Includes

In [6]:
# define the program we will include
program1 = guidance('''List of people:
{{#each people}}- {{this}}
{{/each~}}''')

# note that {{>prog_name}} is the same include syntax as in Handlebars
program2 = guidance('''{{>program1}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

# we can pass program just like any other variable
executed_program2 = program2(program1=program1, people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 
List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

### Flushing caches

In [7]:
# you can flush a cache by calling the clear method
guidance.llms.OpenAI.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# llm = guidance.llms.OpenAI("gpt-4o-mini", caching=False)

### Hidden

In [8]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''
{{~#block hidden=True ~}}
Generate a response to the following email: {{email}}.
Response:{{set 'response' "Me too!"}}
{{~/block~}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}''')

executed_program = program(email='I hate tacos')


Stop program I will show you an email and a response, and you will tell me if it's offensive.
Email: I hate tacos .
Response: Me too!

### Silent execution

In [9]:
# if you want to run a program without displaying the output, you can use the silent=True argument
program = guidance('''
{{~#block hidden=True ~}}
Generate a response to the following email: {{email}}.
Response:{{set 'response' "Me too!"}}
{{~/block~}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
{{set 'answer' "Not offensive"}}
''')

executed_program = program(email='I hate tacos', silent=True)
executed_program['answer']

'Not offensive'

## Generation

### Generating text from an LLM

In [10]:
# we can use the {{gen}} command to generate text from the language model
# note that we used a ~ at the start of the command tag to remove the whitespace before it (just like in Handlebars)
program = guidance('''
{{#system~}}
Complete the sentence
{{~/system}}

{{#user~}}
The best thing about the beach is...
{{~/user}}

{{#assistant~}}
{{~gen 'best' temperature=0.7 max_tokens=7}}
{{~/assistant}}
''')
executed_program = program()

Stop program system Complete the sentence user The best thing about the beach is... assistant the soothing sound of the waves crashing

### Token probabilities from the LLM

In [11]:
program = guidance('''
{{#user~}}
Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:
{{~/user}}

{{#assistant~}}
{{gen "answer" logprobs=True}}
{{~/assistant}}
''', caching=False)
executed_program = program(example='I hate tacos')

Stop program user Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I hate tacos 
Answer: assistant No

In [12]:
# all the variables set by the program are returned as part of the executed program
executed_program['answer_logprobs']

{'No': -0.0009114635}

In [13]:
executed_program['answer']

'No'

### Generating with `n>1`

In [14]:
# the {{gen}} command the n=number argument to generate multiple completions
# only the first completion is used for future context, but the variable set
# by the command is a list of all the completions, and you can interactively
# click through each completion in the notebook visualization
program = guidance('''
{{#system~}}
Complete the sentence
{{~/system}}

{{#user~}}
The best thing about the beach is...
{{~/user}}

{{#assistant~}}
{{~gen 'best' n=3 temperature=1.5 max_tokens=7 logprobs=True}}
{{~/assistant}}
''')

executed_program = program()

Stop program system Complete the sentence user The best thing about the beach is... assistant <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_c5d874970abc4521901708255b6ad8cc(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("c5d874970abc4521901708255b6ad8cc_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("c5d874970abc4521901708255b6ad8cc_" + (i+1)); if (!next_el) { next_el = document.getElementById("c5d874970abc4521901708255b6ad8cc_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_c5d874970abc4521901708255b6ad8cc(this);'>1/3 the soothing sound of waves crashing, the soothing sound of the waves, the soothing sound of the waves,

In [15]:
executed_program["best"]

['the soothing sound of waves crashing,',
 'the soothing sound of the waves,',
 'the soothing sound of the waves,']

### Calling custom user defined functions

In [16]:
# all the built in commands are functions from guidance.library.* but you can also pass in your own functions
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])

# note that we use hidden=True to prevent the {{gen}} command from being included in the output, and instead
# just use the variable it sets as an input to the aggregate function
program = guidance('''
{{#system~}}
Complete the sentence
{{~/system}}

{{#user~}}
The best thing about the beach is...
{{~/user}}

{{#assistant~}}
{{~gen 'best' n=3 temperature=1.5 max_tokens=7 logprobs=True}}
{{~/assistant}}

{{aggregate best}}
''')
executed_program = program(aggregate=aggregate)

Stop program system Complete the sentence user The best thing about the beach is... assistant <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_1dc8c31edff1499281868909473d27ce(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("1dc8c31edff1499281868909473d27ce_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("1dc8c31edff1499281868909473d27ce_" + (i+1)); if (!next_el) { next_el = document.getElementById("1dc8c31edff1499281868909473d27ce_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_1dc8c31edff1499281868909473d27ce(this);'>1/3 the soothing sound of waves crashing, the soothing sound of the waves, the soothing sound of the waves, - the soothing sound of waves crashing,
- the soothing sound of the waves,
- the soothing sound of the waves,

### Await

In [17]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables (so after the await command the variable)
program = guidance('''
{{#user~}}
Generate a response to the following email:
<email>{{email}}.</email>
Response:
{{~/user}}

{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}

{{#user~}}
{{await 'instruction'}}
{{~/user}}

{{#assistant~}}
{{gen 'updated_response'}}
{{~/assistant}}
''', stream=False)

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
program = program(email='Hello there')

Stop program user Generate a response to the following email:
<email> Hello there .</email>
Response: assistant Subject: Re: Hello there

Hi [Name],

Thank you for reaching out! How can I assist you today?

Best regards, 
[Your Name] 
[Your Position] 
[Your Contact Information] user {{await 'instruction'}} {{#assistant~}} 
 {{gen 'updated_response'}} 
 {{~/assistant}}

In [18]:
program2 = program(instruction='Please translate the response above to Portuguese.')

Stop program user Generate a response to the following email:
<email> Hello there .</email>
Response: assistant Subject: Re: Hello there

Hi [Name],

Thank you for reaching out! How can I assist you today?

Best regards, 
[Your Name] 
[Your Position] 
[Your Contact Information] user Please translate the response above to Portuguese. assistant Assunto: Re: Olá

Olá [Nome],

Obrigado por entrar em contato! Como posso ajudá-lo hoje?

Atenciosamente, 
[Seu Nome] 
[Sua Posição] 
[Suas Informações de Contato]

In [19]:
program2 = program(instruction='Please translate the response above to Chinese.')

Stop program user Generate a response to the following email:
<email> Hello there .</email>
Response: assistant Subject: Re: Hello there

Hi [Name],

Thank you for reaching out! How can I assist you today?

Best regards, 
[Your Name] 
[Your Position] 
[Your Contact Information] user Please translate the response above to Chinese. assistant 主题：回复：你好

你好 [姓名]，

感谢你的来信！请问我能如何帮助你？

最好的祝福， 
[你的名字] 
[你的职位] 
[你的联系信息]

## Chat

In [20]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

executed_program = program(conversation_question='What is the meaning of life?')

Stop program system You are a helpful assistant. user What is the meaning of life? assistant The meaning of life is a profound and philosophical question that has been contemplated by thinkers, theologians, and individuals throughout history. Different cultures, religions, and philosophies offer various interpretations:

1. **Religious Perspectives**: Many religions provide specific meanings, often centered around the idea of serving a higher power, following divine commandments, or achieving spiritual enlightenment. For example, in Christianity, the meaning of life may involve loving God and others, while in Buddhism, it may focus on achieving enlightenment and escaping the cycle of suffering.

2. **Philosophical Views**: Philosophers have proposed various theories about the meaning of life. Existentialists, like Jean-Paul Sartre, argue that life has no inherent meaning, and it is up to individuals to create their own purpose. Utilitarianism suggests that the meaning of life could be found in maximizing happiness and reducing suffering.

3. **Personal Meaning**: Many people find meaning through personal experiences, relationships, achievements, and contributions to society. This perspective emphasizes that meaning is subjective and can vary greatly from person to person.

4. **Scientific Perspectives**: From a scientific viewpoint, some may argue that life is a product of evolution and natural processes, and thus, any meaning is constructed by humans rather than inherent in existence itself.

Ultimately, the meaning of life is a deeply personal question, and individuals may find their own answers through reflection, exploration, and experience.

### Multistep

In [21]:
# you can create and guide multi-turn conversations by using a series of role tags
experts = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}

{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
                   
executed_program = experts(query='What is the meaning of life?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant Certainly! Here are three world-class experts who would be great at answering the question about the meaning of life:

1. **Viktor Frankl** - A psychiatrist and Holocaust survivor, Frankl is best known for his book "Man's Search for Meaning," where he explores the importance of finding purpose even in the most difficult circumstances. His experiences and insights into human resilience and meaning make him a profound voice on this topic.

2. **Albert Camus** - A French philosopher and writer, Camus is known for his contributions to existentialism and absurdism. His works, such as "The Myth of Sisyphus," delve into the search for meaning in a seemingly indifferent universe, making him a significant figure in discussions about life's meaning.

3. **Maya Angelou** - An acclaimed poet, memoirist, and civil rights activist, Angelou's reflections on life, love, and the human experience provide deep insights into the meaning of life. Her ability to articulate the complexities of existence through her writing and speeches makes her a powerful voice in this conversation. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: The meaning of life is a profound and multifaceted question that has intrigued humanity for centuries. Perspectives on this topic vary widely, reflecting the complexity of human experience.

Some might argue that the search for meaning is inherently tied to individual purpose and personal experiences. For instance, finding meaning can stem from the pursuit of goals, relationships, and the impact one has on others. This view emphasizes the importance of creating one's own purpose through actions and connections, suggesting that meaning is not given but rather constructed through our choices and experiences.

Others might contend that meaning is found in the acceptance of life's inherent absurdity. In a universe that often appears indifferent, some propose that embracing the absurd can lead to a deeper appreciation of life itself. This perspective encourages individuals to find joy and significance in the present moment, regardless of external circumstances, suggesting that meaning can be derived from the act of living authentically and fully.

Additionally, some may argue that meaning is discovered through suffering and resilience. Life's challenges can lead to profound insights and a greater understanding of oneself and the world. This view posits that even in the darkest times, individuals can find purpose by transcending their suffering and contributing to the greater good, thus transforming pain into a source of strength and meaning.

Ultimately, the meaning of life may not be a singular answer but rather a tapestry woven from diverse experiences, beliefs, and reflections. Each individual must navigate their own journey to uncover what resonates most deeply with them, recognizing that the quest for meaning is as significant as the meaning itself.

### With hidden

In [22]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have guidance controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''')

executed_program = program()

Stop program system You are a helpful assistant. user Is the following joke funny? Why or why not?
 Sure! Here’s a classic one for you:

Why did the scarecrow win an award?

Because he was outstanding in his field! assistant The joke is a classic example of a pun, which is a form of wordplay that relies on the double meaning of words. In this case, "outstanding" can mean both "exceptionally good" and "standing out in a field," which is a literal description of a scarecrow's role.

Whether or not the joke is funny can depend on the audience's taste in humor. Some people enjoy puns and find them clever and amusing, while others may find them cheesy or overused. The simplicity and lightheartedness of the joke make it suitable for a wide range of audiences, including children. Overall, it has a charming quality that can elicit a smile or a groan, which is often the hallmark of a good pun.

### Agents

In [23]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text') hidden=False}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user {{set 'conversation[-1].user_text' (await 'user_text') hidden=False}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text') hidden=False}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [24]:
# as we go through the loop we build up a conversation variable that contains the history of the conversation
# note that the last entry in the conversation variable is empty because the `await` call happens before any
# content is added to the `this` variable that represents the current item in the geneach loop
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {}]

In [25]:
# here we call the agent again and the loop continues, in this case building out a conversation
program = program(user_text = 'What is the meaning of life?')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user What is the meaning of life? assistant The meaning of life is a profound and philosophical question that has been explored by thinkers, theologians, and scientists throughout history. Different cultures, religions, and individuals offer various interpretations. Here are a few perspectives:

1. **Philosophical**: Some philosophers argue that life has no inherent meaning, and it is up to each individual to create their own purpose through experiences, relationships, and personal growth.

2. **Religious**: Many religions provide specific meanings or purposes, often centered around concepts like serving a higher power, achieving enlightenment, or preparing for an afterlife.

3. **Existential**: Existentialists suggest that life is inherently absurd, and individuals must find their own meaning in a chaotic and indifferent universe.

4. **Scientific**: From a biological perspective, the purpose of life can be seen as survival and reproduction, ensuring the continuation of one's genes.

5. **Personal**: Many people find meaning through connections with others, love, creativity, and contributing to society.

Ultimately, the meaning of life can be deeply personal and may vary from person to person. What does it mean to you? user {{set 'conversation[-1].user_text' (await 'user_text') hidden=False}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text') hidden=False}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [26]:
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {'user_text': 'What is the meaning of life?',
  'ai_text': "The meaning of life is a profound and philosophical question that has been explored by thinkers, theologians, and scientists throughout history. Different cultures, religions, and individuals offer various interpretations. Here are a few perspectives:\n\n1. **Philosophical**: Some philosophers argue that life has no inherent meaning, and it is up to each individual to create their own purpose through experiences, relationships, and personal growth.\n\n2. **Religious**: Many religions provide specific meanings or purposes, often centered around concepts like serving a higher power, achieving enlightenment, or preparing for an afterlife.\n\n3. **Existential**: Existentialists suggest that life is inherently absurd, and individuals must find their own meaning in a chaotic and indifferent universe.\n\n4. **Scientific**: From a biological perspective, the

### Using tools

The example below uses a search engine (or a mock of one) to answer user questions. The whole system is defined in a single `guidance` program, but you could also break it into multiple programs and `await` external calls if you don't want the guidance program to control the whole process.

In [27]:
def is_search(completion):
    return '<search>' in completion

def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
        'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
        {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
        'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
        {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
        'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]

search_demo = guidance('''Seach results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}''')

demo_results = [
    {'title': 'OpenAI - Wikipedia', 'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
    {'title': 'About - OpenAI', 'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'}, 
    {'title': 'Ilya Sutskever | Stanford HAI', 'snippet': '''Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.'''}
]

s = search_demo(results=demo_results)

practice_round = [
    {'role': 'user', 'content' : 'Who are the founders of OpenAI?'},
    {'role': 'assistant', 'content': '<search>Who are the founders of OpenAI</search>'},
    {'role': 'user', 'content': str(search_demo(results=demo_results))},
    {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.'},
]

program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}

{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}

{{#each practice}}
{{#if this.role=="user"}}
{{#user}}{{this.content}}{{/user}}
{{else}}
{{#assistant}}{{this.content}}{{/assistant}}
{{/if}}
{{/each}}

{{#user~}}
That was great, now let's do another one.
{{~/user}}

{{#assistant~}}
Sounds good
{{~/assistant}}

{{#user~}}
{{user_query}}
{{~/user}}

{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}

{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
''')

query = "What is Facebook's stock price right now?"

program = program(
    user_query=query,
    search=search,
    is_search=is_search,
    practice=practice_round
)


Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round 
 user Who are the founders of OpenAI? 

 assistant <search>Who are the founders of OpenAI</search> 

 user Seach results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result> 

 assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. 
 user That was great, now let's do another one. assistant Sounds good user What is Facebook's stock price right now? assistant <search>Facebook stock price </search> user Search results: 
<result>
 How do I cancel a Subscription? | Facebook Help Center 
 To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ... 
</result>
<result>
 News | FACEBOOK Stock Price Today | Analyst Opinions - Insider 
 Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr... 
</result>
<result>
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ... 
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved 
</result> assistant Facebook, now known as Meta Platforms Inc (NASDAQ: META), has a stock price of $197.81, which is up by $2.20 (1.12%).

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>